# Notebook for exporting screenshots for annotation

In [11]:
import os
import json
import shutil

from ultralytics import YOLO
from PIL import Image

import sys
sys.path.insert(0, "../../hindsight/hindsight_server/")

from db import HindsightDB
from utils import make_dir

In [12]:
db = HindsightDB()

In [13]:
frames = db.get_frames(impute_applications=False)
frames = frames.loc[frames['application'] == "Twitter"]

In [14]:
sel_frames = frames.sample(n=100)

In [15]:
dest_dir = "../data/annotations/twitter/rand_upload_2/"
make_dir(dest_dir)
for f in sel_frames['path']:
    dest_f = os.path.join(dest_dir, os.path.basename(f))
    shutil.copy(f, dest_dir)

# Run prediction model on frames

In [16]:
trained_model = YOLO("./runs/detect/train15/weights/best.pt")

In [17]:
pred_images_dir = dest_dir 
images = list()
for f in os.listdir(pred_images_dir):
    im = Image.open(os.path.join(pred_images_dir, f))
    images.append(im)

In [18]:
results = trained_model(images)


0: 640x320 2 tweets, 138.7ms
1: 640x320 3 tweets, 138.7ms
2: 640x320 1 more_posted, 3 tweets, 138.7ms
3: 640x320 3 tweets, 138.7ms
4: 640x320 2 tweets, 138.7ms
5: 640x320 2 tweets, 138.7ms
6: 640x320 3 tweets, 138.7ms
7: 640x320 2 tweets, 138.7ms
8: 640x320 1 plus_button, 8 tweets, 138.7ms
9: 640x320 1 more_posted, 3 tweets, 138.7ms
10: 640x320 4 tweets, 138.7ms
11: 640x320 3 tweets, 138.7ms
12: 640x320 1 more_posted, 3 tweets, 138.7ms
13: 640x320 4 tweets, 138.7ms
14: 640x320 3 tweets, 1 twitter_top_menu, 138.7ms
15: 640x320 3 tweets, 138.7ms
16: 640x320 1 more_posted, 1 plus_button, 3 tweets, 1 twitter_bottom_menu, 1 twitter_top_menu, 138.7ms
17: 640x320 1 more_posted, 3 tweets, 138.7ms
18: 640x320 3 tweets, 138.7ms
19: 640x320 1 plus_button, 3 tweets, 1 twitter_bottom_menu, 1 twitter_top_menu, 138.7ms
20: 640x320 3 tweets, 138.7ms
21: 640x320 1 more_posted, 1 plus_button, 3 tweets, 1 twitter_bottom_menu, 1 twitter_top_menu, 138.7ms
22: 640x320 1 plus_button, 4 tweets, 138.7ms
23: 6

In [19]:
result = results[0]

In [20]:
for i, box in enumerate(result.boxes):
    print(box.xyxyn)

tensor([[0.0000, 0.0513, 1.0000, 0.3631]])
tensor([[0.0000, 0.3648, 1.0000, 0.9820]])


In [21]:
box

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([10.])
conf: tensor([0.9864])
data: tensor([[0.0000e+00, 8.7553e+02, 1.0880e+03, 2.3567e+03, 9.8641e-01, 1.0000e+01]])
id: None
is_track: False
orig_shape: (2400, 1088)
shape: torch.Size([1, 6])
xywh: tensor([[ 544.0000, 1616.1276, 1088.0000, 1481.1868]])
xywhn: tensor([[0.5000, 0.6734, 1.0000, 0.6172]])
xyxy: tensor([[   0.0000,  875.5342, 1088.0000, 2356.7209]])
xyxyn: tensor([[0.0000, 0.3648, 1.0000, 0.9820]])

In [60]:
all_preds = list()

In [61]:
for result in results:
    image_preds_d = {}
    image_path = f"/data/local-files/?d={result.path}"
    image_preds_d['data'] = {"image" : image_path}
    predictions_d = {"model_version": "train13", "score": 0.55}
    org_width = result.orig_shape[1]
    org_height = result.orig_shape[0]
    result_d_template = {"type": "rectanglelabels",        
            "from_name": "label", "to_name": "image",
            "original_width": org_width, "original_height": org_height,
            "image_rotation": 0}
    converted_results = list()
    for i, box in enumerate(result.boxes):
        result_d = result_d_template.copy()
        result_d['id'] = f"result{i}"
        value_d = {"rotation" : 0,
                "x" : ((float(box.xyxyn[0][0])) * 100),
                "y" : (float(box.xyxyn[0][1])) * 100,
                "width" : ((float(box.xyxyn[0][2]) - float(box.xyxyn[0][0])) * 100), 
                "height": (float(box.xyxyn[0][3]) - float(box.xyxyn[0][1])) * 100, 
                "rectanglelabels": [result.names[int(box.cls[0])]]}
        result_d['value'] = value_d
        converted_results.append(result_d)
        
    predictions_d["result"] = converted_results
    image_preds_d['predictions'] = [predictions_d]
    all_preds.append(image_preds_d)

In [62]:
with open("rand_upload_2.json", 'w') as outfile:
    json.dump(all_preds, outfile)

In [1]:
import sys
sys.path.insert(0, "../")

from annotation_db import HindsightAnnotationsDB

In [2]:
annotation_db = HindsightAnnotationsDB()

In [24]:
annotations = annotation_db.get_annotations()

In [25]:
annotations

,id,frame_id,x,y,w,h,rotation,label,conf,model_name,model_version
